Paso 1: Instalar la API de Kaggle

pip install kaggle

Paso 2: Obtener tu API Key de Kaggle

En la sección "API", haz clic en Create New API Token. Esto descargará un archivo llamado kaggle.json.

Paso 3: Configurar la API Key

- En sistemas Unix/Linux/Mac:

    (en bash)

    mkdir ~/.kaggle

    mv kaggle.json ~/.kaggle/

    chmod 600 ~/.kaggle/kaggle.json

- En Windows:

    Crea la carpeta .kaggle en C:\Users\<tu_nombre_de_usuario>\.

    Mueve el archivo kaggle.json a esa carpeta.

Paso 4: Buscar el Dataset en Kaggle

Paso 5: Descargar el Dataset

kaggle datasets download -d <autor>/<nombre-dataset>

(Esto descargará un archivo .zip en tu directorio actual.)

Paso 6: Extraer el Dataset

In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array


from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

from PIL import Image
import matplotlib.pyplot as plt
import os

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


from keras.optimizers import Adam
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Rutas a las carpetas
train_dir = "../dogs-vs-cats/train"

# Filtrar nombres de archivos de perros y gatos
dog_files = [os.path.join(train_dir, f) for f in os.listdir(train_dir) if f.startswith("dog")]
cat_files = [os.path.join(train_dir, f) for f in os.listdir(train_dir) if f.startswith("cat")]

# Función para cargar, redimensionar y visualizar imágenes
def plot_images_pillow(image_paths, title, size=(200, 200)):
    plt.figure(figsize=(10, 10))
    for i, img_path in enumerate(image_paths[:9]):  # Mostrar las primeras 9 imágenes
        try:
            img = Image.open(img_path)  # Cargar imagen
            img = img.resize(size)  # Redimensionar
            img = img.convert("RGB")
            plt.subplot(3, 3, i + 1)  # Crear una cuadrícula de 3x3
            plt.imshow(img)
            plt.axis("off")    
            plt.title(f"Img {i+1}", fontsize=10)        
        except Exception as e:
            print(f"Error al cargar la imagen {img_path}: {e}")
            continue


    plt.suptitle(title, fontsize=16)
    plt.show()

# Mostrar las primeras 9 imágenes de perros
plot_images_pillow(dog_files, "Perros", size=(200, 200))

# Mostrar las primeras 9 imágenes de gatos
plot_images_pillow(cat_files, "Gatos", size=(200, 200))

In [ ]:
# Directorio de datos
train_dir = "../dogs-vs-cats/train"

# Dimensiones de las imágenes
img_size = (200, 200)

# Inicializar listas para imágenes y etiquetas
images = []
labels = []

# Cargar imágenes y asignar etiquetas
for filename in os.listdir(train_dir):
    label = 1 if filename.startswith("dog") else 0  # 1 para perros, 0 para gatos
    img_path = os.path.join(train_dir, filename)
    
    try:
        img = load_img(img_path, target_size=img_size)  # Cargar y redimensionar
        img_array = img_to_array(img) / 255.0  # Normalizar a rango [0, 1]
        images.append(img_array)
        labels.append(label)
    except Exception as e:
        print(f"Error al procesar {filename}: {e}")

# Convertir listas a arrays de NumPy
images = np.array(images)
labels = np.array(labels)

# Mostrar información sobre los datos cargados
print(f"Total de imágenes cargadas: {images.shape[0]}")
print(f"Dimensiones de las imágenes: {images.shape[1:]}")  # (200, 200, 3)
print(f"Distribución de etiquetas: {np.unique(labels, return_counts=True)}")

1. Cargar imágenes con load_img:

    - Redimensionamos cada imagen a 200x200 píxeles directamente al cargarla.

2. Convertir a arrays:

    - Usamos img_to_array para convertir la imagen en un array NumPy.

    - Normalizamos los valores de píxeles a [0, 1] dividiendo entre 255.

3. Etiquetado:

    - Los nombres de los archivos (cat y dog) determinan las etiquetas (0 o 1).

4. Convertir a Arrays:

    - Finalmente, convertimos las listas de imágenes y etiquetas a arrays NumPy para facilitar el procesamiento posterior.

5. Resultado:

    - images tiene la forma (25000, 200, 200, 3) (25,000 imágenes de 200x200 píxeles con 3 canales de color).

    - labels tiene la forma (25000,) con las etiquetas correspondientes.

    ---

In [ ]:
# Directorio de entrenamiento y prueba
train_dir = "../dogs-vs-cats/train"
test_dir = "../dogs-vs-cats/test1"

# Crear un DataFrame para las imágenes de entrenamiento
# Convertir etiquetas numéricas a strings
train_files = os.listdir(train_dir)
train_labels = ['dog' if 'dog' in f else 'cat' for f in train_files]

# Crear DataFrame con etiquetas en formato string
train_df = pd.DataFrame({
    'filename': [os.path.join(train_dir, f) for f in train_files],
    'class': train_labels
})

# Configurar el ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Separar 20% de los datos para validación
)

# Generadores de datos de entrenamiento y validación
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
# Crear un DataFrame para las imágenes de prueba
test_files = os.listdir(test_dir)
test_df = pd.DataFrame({
    'filename': [os.path.join(test_dir, f) for f in test_files]
})

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,  # Sin etiquetas
    shuffle=False
)

## Paso 3: Construye una RNA

Cualquier clasificador que se ajuste a este problema tendrá que ser robusto porque algunas imágenes muestran al gato o al perro en una esquina o tal vez a 2 gatos o perros en la misma foto. Si has podido investigar algunas de las implementaciones de los ganadores de otras competiciones también relacionadas con imágenes, verás que VGG16 es una arquitectura de CNN utilizada para ganar la competencia de Kaggle ILSVR (Imagenet) en 2014. Se considera una de las arquitecturas de modelos de visión con mejores resultados hasta la fecha.

Utiliza la siguiente arquitectura de prueba:

In [ ]:
# Definir el modelo secuencial
model = Sequential()

# Bloque 1
model.add(Conv2D(input_shape=(224, 224, 3), filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Bloque 2
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Bloque 3
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Bloque 4
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Bloque 5
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Capas completamente conectadas
model.add(Flatten())
model.add(Dense(units=4096, activation="relu"))
model.add(Dense(units=4096, activation="relu"))


model.add(Dense(units=1, activation="sigmoid"))  # Capa de salida con una neurona

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

In [ ]:
import scipy
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=1 ,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("../models/vgg16_1.h5", monitor = "val_accuracy", verbose = 1, save_best_only = True, save_weights_only = False, mode = "auto")
early = EarlyStopping(monitor = "val_accuracy", patience = 3, verbose = 1, mode = "auto")
hist = model.fit(train_generator, steps_per_epoch = 100, validation_data = test_generator, validation_steps = 10, epochs = 3, callbacks = [checkpoint, early])

In [ ]:
# Trazar los resultados
plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])

# Configurar el diseño del gráfico
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "Loss", "Validation Loss"])

# Trazar
plt.show()

In [ ]:
from keras.preprocessing import image
from keras.models import load_model

img = image.load_img("../dogs-vs-cats/test1/9.jpg", target_size = (200, 200))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis = 0)
saved_model = load_model("../models/vgg16_1.h5")
output = saved_model.predict(img)
if output[0][0] > output[0][1]:
    print("cat")
else:
    print("dog")